In [9]:
import pandas as pd
import os
os.chdir("I:/WeatherData/Weather Data/OPSD")
df = pd.read_csv('weather_data.csv')
df.head(3)

,utc_timestamp,BG_temperature,BG_radiation_direct_horizontal,BG_radiation_diffuse_horizontal,EE_temperature,EE_radiation_direct_horizontal,EE_radiation_diffuse_horizontal,ES_temperature,ES_radiation_direct_horizontal,ES_radiation_diffuse_horizontal,...,LU_radiation_direct_horizontal,LU_radiation_diffuse_horizontal,NL_windspeed_10m,NL_temperature,NL_radiation_direct_horizontal,NL_radiation_diffuse_horizontal,PL_windspeed_10m,PL_temperature,PL_radiation_direct_horizontal,PL_radiation_diffuse_horizontal
0,1980-01-01T00:00:00Z,4.664,0.0,0.0,-7.060,0.0,0.0,8.066,0.0,0.0,...,0.0,0.0,7.52,2.382,0.0,0.0,3.57,-3.721,0.0,0.0
1,1980-01-01T01:00:00Z,4.052,0.0,0.0,-7.341,0.0,0.0,7.960,0.0,0.0,...,0.0,0.0,7.28,2.236,0.0,0.0,3.50,-3.806,0.0,0.0
2,1980-01-01T02:00:00Z,3.581,0.0,0.0,-7.591,0.0,0.0,8.008,0.0,0.0,...,0.0,0.0,6.95,2.086,0.0,0.0,3.49,-3.868,0.0,0.0


In [2]:
#Initializing PySpark
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

# #Spark Config
conf = SparkConf().setAppName("sample_app")
sc = SparkContext(conf=conf)

from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [3]:
df = spark.read.csv(
    'weather_data.csv', header=True, inferSchema=True
)

In [4]:
df.createOrReplaceTempView('df')

In [7]:
import datetime
from pyspark.sql.functions import year, month, dayofmonth

In [8]:
select min(date_before), count(utc_timestamp) + 1
from (select strftime('%d', utc_timestamp) - lag(strftime('%d', utc_timestamp)) over (order by utc_timestamp) diff
           , strftime('%m', utc_timestamp)                                                                    month
           , strftime('%Y', utc_timestamp)                                                                    year
           , utc_timestamp
           , lag(utc_timestamp) over (order by utc_timestamp)                                                 date_before
      FROM (
               SELECT utc_timestamp, AVG(GB_temperature) as avgt
               FROM weather_data
               GROUP BY strftime('%Y-%m-%d', utc_timestamp)
               HAVING avgt > 20
           )
      order by utc_timestamp)
where diff = 1
group by month, year;

ParseException: 
extraneous input 'diff' expecting {')', ','}(line 2, pos 101)

== SQL ==
select min(date_before), count(utc_timestamp) + 1
from (select dayofmonth(utc_timestamp) - lag(dayofmonth(utc_timestamp) over (order by utc_timestamp) diff
-----------------------------------------------------------------------------------------------------^^^
           , month(utc_timestamp)                                                                    month
           , year(utc_timestamp)                                                                    year
           , utc_timestamp
           , lag(utc_timestamp) over (order by utc_timestamp)                                                 date_before
      FROM (
               SELECT utc_timestamp, AVG(GB_temperature) as avgt
               FROM weather_data
               GROUP BY (utc_timestamp)
               HAVING avgt > 20
           )
      order by utc_timestamp)
where diff = 1
group by month, year


In [8]:
df.show(3)

+--------------------+--------------+------------------------------+-------------------------------+--------------+------------------------------+-------------------------------+--------------+------------------------------+-------------------------------+--------------+------------------------------+-------------------------------+--------------+------------------------------+-------------------------------+--------------+------------------------------+-------------------------------+--------------+------------------------------+-------------------------------+--------------+------------------------------+-------------------------------+--------------+------------------------------+-------------------------------+--------------+------------------------------+-------------------------------+--------------+------------------------------+-------------------------------+--------------+------------------------------+-------------------------------+--------------+---------------------------